<a href="https://colab.research.google.com/github/HikariZasureiya/HLV25FinanceScript/blob/main/HLV_finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade gspread


In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [2]:
def findit(rolltostream :dict ,sheetno :int , nameworksheet ,sheetname :str):
    values = nameworksheet.get_all_values()
    df = pd.DataFrame(values[1:], columns=values[0])
    tylis = pd.DataFrame(df)
    try:
      for idx, roll in enumerate(tylis['ROLL_NUMBER']):
        roll = str(roll).strip()
        if pd.notna(roll) and roll in rolltostream:
          print(f"Sheet {sheetno+1} ➜", idx, roll)

    except Exception as e:
        print(e)
        print(f'error in sheet {sheetno} , {sheetname}')


In [3]:
def writeit(rolltostream :dict ,sheetno :int , nameworksheet ,sheetname :str):
    values = nameworksheet.get_all_values()
    df = pd.DataFrame(values[1:], columns=values[0])
    tylis = pd.DataFrame(df)


    try:
      for idx, roll in enumerate(tylis['ROLL_NUMBER']):
        roll = str(roll).strip()
        if pd.notna(roll) and roll in rolltostream:
          row_num = idx+2
          nameworksheet.update_cell(row_num, 3, 'Y')
          nameworksheet.update_cell(row_num, 4, rolltostream[roll][1])

    except Exception as e:
        print(e)
        print(f'error in sheet {sheetno} , {sheetname}')
        # wb.close()

In [4]:
import pandas as pd
from openpyxl import load_workbook


#opens hlv'25 collection spreadsheet
sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1s-E2MnVyXGZSobcnaZ4SqJba0Tao4B2Zdoux6-vu4ZU/edit?gid=1018150263#gid=1018150263")
worksheet = sheet.worksheets()[0]
data = worksheet.get_all_records()
fin = pd.DataFrame(data)



rolltostream ={}

for idx , roll in enumerate(fin['University Roll No']):
    temp = ''
    if(fin.loc[idx , 'Food Preference'] == 'Non Veg'):
        temp = 'NON-VEG'
    elif(fin.loc[idx , 'Food Preference'] == 'Veg'):
        temp = 'VEG'
    rolltostream[str(roll)] = [str(fin.loc[idx , 'Stream']) , temp]

print(rolltostream , len(rolltostream))

{'13000123103': ['CSE', 'NON-VEG'], '13000123101': ['CSE', 'NON-VEG'], '13000123117': ['CSE', 'NON-VEG'], '13000124121': ['CSE', 'NON-VEG'], '13000123124': ['CSE', 'NON-VEG'], '13000123095': ['CSE', 'NON-VEG'], '13000123067': ['CSE', 'NON-VEG'], '13000123113': ['CSE', 'NON-VEG'], '13031124051': ['CSBS', 'NON-VEG'], '13000123110': ['CSE', 'VEG'], '13000123090': ['CSE', 'NON-VEG'], '13000123105': ['CSE', 'NON-VEG'], '13000124159': ['CSE', 'VEG'], '13000123018': ['CSE', 'VEG'], '13000123052': ['CSE', 'NON-VEG'], '13000122039': ['CSE', 'NON-VEG']} 16


In [6]:
namelist = gc.open_by_url("https://docs.google.com/spreadsheets/d/1We74wo0hkN73b2Wa3hqtgoqnNAGFtPjls4_NiwkRGwc/edit?gid=1031123412#gid=1031123412")
nameworksheet = namelist.worksheets()
sheetno = len(nameworksheet)

for i in range(0,sheetno):
  findit(rolltostream , i , nameworksheet[i] , nameworksheet[i].title)

# ''' uncomment and execute to modify sheet '''

# for i in range(0,sheetno):
#   writeit(rolltostream , i , nameworksheet[i] , nameworksheet[i].title)



Sheet 3 ➜ 2 13000124121
Sheet 4 ➜ 17 13000123018
Sheet 4 ➜ 51 13000123052
Sheet 5 ➜ 17 13000123018
Sheet 5 ➜ 50 13000123052
Sheet 6 ➜ 45 13000122039
